In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_cleancoll = pd.read_csv('cleanVIF_loan_test4.csv')

In [3]:
df_cleancoll.head()

,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,...,mths_since_last_record,pub_rec,revol_bal,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,flag_bad_loan
0,0,0,0,0,0,0,1,0,0,0,...,0.0,0.0,36638.0,0.0,885.46,0.0,0.0,114834.0,59900.0,0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,7967.0,0.0,333.14,0.0,0.0,14123.0,15100.0,0
2,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,13168.0,0.0,368.45,0.0,0.0,267646.0,61100.0,0
3,0,0,0,0,0,0,0,0,0,1,...,33.0,2.0,5572.0,0.0,119.17,0.0,15386.0,13605.0,8100.0,0
4,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,11431.0,0.0,476.30,0.0,1514.0,272492.0,15400.0,0


## 5. MODELLING

#### Train Test Split

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df_cleancoll.drop('flag_bad_loan',axis=1)
y = df_cleancoll['flag_bad_loan']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [8]:
data = df_cleancoll.drop('flag_bad_loan', axis =1)
target = df_cleancoll['flag_bad_loan']

#### Logistic Regression

Normal X_train

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
predictions_train = logmodel.predict(X_train)
predictions_test = logmodel.predict(X_test)
PredictProba = logmodel.predict_proba(X_test)

In [22]:
adjustpredictions = []
for pred in PredictProba[:,0]:
    if (pred>0.30):
        adjustpredictions.append(0)
    else:
        adjustpredictions.append(1) 

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, log_loss

In [24]:
print(classification_report(y_train,predictions_train))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98    538461
           1       1.00      0.28      0.44     33153

    accuracy                           0.96    571614
   macro avg       0.98      0.64      0.71    571614
weighted avg       0.96      0.96      0.95    571614



In [25]:
print(classification_report(y_test,predictions_test))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98    230233
           1       1.00      0.29      0.45     14745

    accuracy                           0.96    244978
   macro avg       0.98      0.64      0.71    244978
weighted avg       0.96      0.96      0.95    244978



In [26]:
print(classification_report(y_test,adjustpredictions))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98    230233
           1       1.00      0.26      0.41     14745

    accuracy                           0.96    244978
   macro avg       0.98      0.63      0.70    244978
weighted avg       0.96      0.96      0.94    244978



Oversampled X_train

In [27]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy='minority')

oversampled_trainX, oversampled_trainY = sm.fit_sample(X_train, y_train)
oversampled_train = pd.concat([pd.DataFrame(oversampled_trainY), pd.DataFrame(oversampled_trainX)], axis =1)

In [28]:
logmodel_os = LogisticRegression()
logmodel_os.fit(oversampled_trainX, oversampled_trainY)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
logmodel_pred_ostrain = logmodel_os.predict(oversampled_trainX)
logmodel_pred_ostest = logmodel_os.predict(X_test)
logmodel_predproba_ostest = logmodel_os.predict_proba(X_test)

In [41]:
adjustpredictions_os = []
for pred in logmodel_predproba_ostest[:,0]:
    if (pred>0.05):
        adjustpredictions_os.append(0)
    else:
        adjustpredictions_os.append(1) 

In [42]:
print(classification_report(oversampled_trainY,logmodel_pred_ostrain))

              precision    recall  f1-score   support

           0       0.70      0.82      0.76    538461
           1       0.78      0.65      0.71    538461

    accuracy                           0.74   1076922
   macro avg       0.74      0.74      0.74   1076922
weighted avg       0.74      0.74      0.74   1076922



In [43]:
print(classification_report(y_test,logmodel_pred_ostest))

              precision    recall  f1-score   support

           0       0.96      0.82      0.89    230233
           1       0.16      0.52      0.24     14745

    accuracy                           0.80    244978
   macro avg       0.56      0.67      0.56    244978
weighted avg       0.92      0.80      0.85    244978



In [44]:
print(classification_report(y_test,adjustpredictions_os))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98    230233
           1       1.00      0.29      0.46     14745

    accuracy                           0.96    244978
   macro avg       0.98      0.65      0.72    244978
weighted avg       0.96      0.96      0.95    244978



# 6. EVALUATION

## K-Fold Model valuation based on iteration

In [45]:
data = df_cleancoll.drop('flag_bad_loan', axis =1)
target = df_cleancoll['flag_bad_loan']

In [54]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score, f1_score, log_loss, matthews_corrcoef

def calc_train_error(X_train, y_train, model):
    predictions =[]
    predictProba = model.predict_proba(X_train)
    for pred in predictProba[:,0]:
        if(pred > 0.10):
            predictions.append(0)
        else:
            predictions.append(1)  
    matt = matthews_corrcoef(y_train, predictions)
    f1 = f1_score(y_train, predictions, average ='macro')
    report = classification_report(y_train, predictions)
    roc_auc = roc_auc_score(y_train, predictProba[:, 1])
    accuracy = accuracy_score(y_train, predictions)
    confMatrix = confusion_matrix(y_train, predictions)
    logloss = log_loss(y_train,predictProba)
    return{
        'report' : report, 
        'matthew' : matt,
        'f1' : f1,
        'roc': roc_auc,
        'accuracy': accuracy,
        'confusion': confMatrix,
        'logloss' : logloss
    }

def calc_validation_error(X_test, y_test, model):
    predictions = []
    predictProba = model.predict_proba(X_test)
    for pred in predictProba[:,0]:
        if(pred > 0.10):
            predictions.append(0)
        else:
            predictions.append(1)   
    
    matt = matthews_corrcoef(y_test, predictions)
    f1 = f1_score(y_test, predictions, average ='macro')
    report = classification_report(y_test, predictions)
    roc_auc = roc_auc_score(y_test, predictProba[:, 1])
    accuracy = accuracy_score(y_test, predictions)
    confMatrix = confusion_matrix(y_test, predictions)
    logloss = log_loss(y_test,predictProba)
    return{
        'report' : report, 
        'matthew' : matt,
        'f1' : f1,
        'roc': roc_auc,
        'accuracy': accuracy,
        'confusion': confMatrix,
        'logloss' : logloss
    }

def calc_metrics(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    train_error = calc_train_error(X_train, y_train, model)
    validation_error = calc_validation_error(X_test, y_test, model)
    return train_error, validation_error

In [55]:
from sklearn.model_selection import KFold
K = 5
kf =KFold(n_splits = K, shuffle = True, random_state = 42)

In [56]:
train_errors = []
validation_errors = []
for train_index, val_index in kf.split(data, target):
    
    #split data
    X_train, X_val = data.iloc[train_index], data.iloc[val_index]
    y_train, y_val = target.iloc[train_index], target.iloc[val_index]
    
    #instantiate model
    logmodel = LogisticRegression()
    
    #calculate errors
    train_error, val_error = calc_metrics(X_train, y_train, X_val, y_val, logmodel)
    
    #append to appropiate list
    train_errors.append(train_error)
    validation_errors.append(val_error)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regre

### Model Summary

In [57]:
matrix = []
for i, j in zip(train_errors, validation_errors):
    matrix.append([i['accuracy'], j['accuracy'], i['roc'],
                   j['roc'], i['f1'], j['f1'], i['matthew'], j['matthew'], i['logloss'], j['logloss']])
calc_matrix = pd.DataFrame(matrix, columns = ['Train Accuracy', 
                                              'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score', 
                                'Test F1 Score', 'Train Matthews Corr Coef',
                                              'Test Matthews Corr Coef', 'Train Log Loss', 'Test Log Loss'])
average = []
for i in calc_matrix.columns:
    average.append(calc_matrix[i].mean())
    
average_mat = pd.DataFrame(average).T
average_mat.columns = ['Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score', 
                                'Test F1 Score', 'Train Matthews Corr Coef',
                       'Test Matthews Corr Coef' , 'Train Log Loss', 'Test Log Loss']
index_label = []
for i in range(1, len(calc_matrix)+1):
    index_label.append(f'{i} Iteration')
index_label.append('Average')    
calculation = pd.concat([calc_matrix, average_mat])
calculation.index = index_label
calculation

,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthews Corr Coef,Train Log Loss,Test Log Loss
1 Iteration,0.954959,0.954267,0.744431,0.744480,0.676412,0.669905,0.470203,0.460269,0.181361,0.182063
2 Iteration,0.954693,0.955749,0.745098,0.741699,0.675900,0.676780,0.469458,0.470643,0.181637,0.180161
3 Iteration,0.954769,0.954904,0.743407,0.747239,0.674463,0.676306,0.467231,0.470050,0.181972,0.179780
4 Iteration,0.954745,0.954781,0.743683,0.745887,0.674407,0.673992,0.467148,0.466501,0.181804,0.181577
5 Iteration,0.954875,0.954365,0.744725,0.741900,0.673655,0.678105,0.465966,0.472904,0.181112,0.184390
Average,0.954808,0.954813,0.744269,0.744241,0.674967,0.675017,0.468001,0.468073,0.181577,0.181594
